In [ ]:
import re
import pandas as pd
import nest_asyncio
from telethon import TelegramClient

nest_asyncio.apply()  

# Telegram credentials
api_id = *12345
api_hash = "***"
phone = "***"


CHANNEL_USERNAME = "atamekenbusiness_qaz"
MAX_MESSAGES = 10000
TARGET_SENTENCES = 400
OUTPUT_FILE = "Kazakh_Cyrillic_400.txt"


def clean_sentence(text):
    if not text:
        return None
    blocked_words = [
        "instagram", "telegram", "youtube",
        "подписывайся", "subscribe", "подписка", "жазылыңыз",
        "@", "http", ".com", ".uz", ".ru", ".org"
    ]
    if any(w.lower() in text.lower() for w in blocked_words):
        return None

    text = re.sub(r"[\*\^\~\|\>\<]", "", text)
    text = re.sub(r"[^\w\s.,!?-]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    if len(text.split()) < 3:
        return None
    return text


async def fetch_messages():
    sentences = []
    async with TelegramClient(phone, api_id, api_hash) as client:
        await client.start(phone=phone)
        print(f"📥 Fetching messages from {CHANNEL_USERNAME}...")

        async for msg in client.iter_messages(CHANNEL_USERNAME, limit=MAX_MESSAGES):
            if msg.message:
                cleaned = clean_sentence(msg.message)
                if cleaned:
                    sentences.append(cleaned)
            if len(sentences) >= TARGET_SENTENCES:
                break

    print(f"✅ Collected {len(sentences)} clean sentences.")
    return sentences


async def main():
    sentences = await fetch_messages()
    df = pd.DataFrame(sentences, columns=["sentence"])
    df["sentence"].to_csv(OUTPUT_FILE, index=False, header=False, encoding="utf-8")
    print(f"💾 Saved {len(sentences)} sentences to {OUTPUT_FILE}")


await main()


#Signed in successfully as ***; remember to not break the ToS or you will risk an account ban!
#📥 Fetching messages from atamekenbusiness_qaz...
#✅ Collected 400 clean sentences.
#💾 Saved 400 sentences to Kazakh_Cyrillic_400.txt